# CrysFML: Neutrons vs X-rays

In [1]:
# Misc
import sys, os
import numpy as np

# CrysFML
import CFML_api

# Vizualization
import py3Dmol
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

In [2]:
output_notebook()
FIGURE_WIDTH = 700
FIGURE_HEIGHT = 300

Loading BokehJS ...

In [3]:
def diffraction_pattern_xy(space_group, cell, atom_list, job_info):
    reflection_list = CFML_api.ReflectionList(cell, space_group, 
                                              True, 
                                              job_info)
    reflection_list.compute_structure_factors(space_group, 
                                              atom_list, 
                                              job_info)
    diffraction_pattern = CFML_api.DiffractionPattern(job_info,
                                                  reflection_list, 
                                                  cell.reciprocal_cell_vol)
    return diffraction_pattern.x, diffraction_pattern.ycalc

In [4]:
def set_default_parameters(job_info):
    job_info.range_2theta = (10.0, 155.45)
    job_info.theta_step = 0.05
    job_info.lambdas = (1.9122, 1.9122)
    job_info.lambda_ratio = 1.0
    job_info.u_resolution = 0.1421
    job_info.v_resolution = -0.4181
    job_info.w_resolution = 0.3864
    job_info.x_resolution = 0.0
    job_info.y_resolution = 0.0904

## Define model

### Create space group object

In [5]:
space_group = CFML_api.SpaceGroup('P n m a')
#space_group.print_description()

### Create unit cell object

In [6]:
lengths = np.asarray([8.477,5.396,6.957], dtype='float32')
angles = np.asarray([90,90,90], dtype='float32')
unit_cell = CFML_api.Cell(lengths, angles)
#unit_cell.print_description()

### Create list of atoms

In [7]:
cif_str = '''
loop_
 _atom_site_label
 _atom_site_type_symbol
 _atom_site_occupancy
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_adp_type
 _atom_site_U_iso_or_equiv
  Pb  Pb  1.0  0.1880  0.25    0.1668  Uiso  0.0169
  S   S   1.0  0.0650  0.25    0.6850  Uiso  0.0025
  O1  O   1.0  0.9080  0.25    0.5950  Uiso  0.0246
  O2  O   1.0  0.1935  0.25    0.5432  Uiso  0.0180
  O3  O   1.0  0.0814  0.0274  0.8085  Uiso  0.0165
'''
atom_list = CFML_api.AtomList(cif_str.split('\n'))
atom_list.set_mult_occ_cif(space_group)
#atom_list.print_description()

### Create job definition object

In [8]:
cfl_str = '''
Title SrTiO3
'''
job_info = CFML_api.JobInfo(cfl_str.split('\n'))
job_info.print_description()

SRTIO3
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases: SRTIO3                                                                                                                          
Name of phases: SRTIO3                                                                                                                          
Lambda range: (1.5405000448226929, 1.5405000448226929)
Lambda ratio: 0.0
Range 2theta: (0.0, 120.0)
Range sin(theta)/lambda: (0.0, 0.5621716380119324)


### Modify job definition object

In [9]:
job_info.pattern_type = "NEUT_2THE"
set_default_parameters(job_info)
job_info.print_description()

SRTIO3
Number of patterns: 1
Type of pattern: NEUT_2THE       
Number of phases: 1
Name of phases: SRTIO3                                                                                                                          
Name of phases: SRTIO3                                                                                                                          
Lambda range: (1.9121999740600586, 1.9121999740600586)
Lambda ratio: 1.0
Range 2theta: (10.0, 155.4499969482422)
Range sin(theta)/lambda: (0.05657626688480377, 0.6342995166778564)


## Calculate neutron powder diffraction pattern

### Plot calculated pattern

In [10]:
x_data, y_data_neutrons = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons, legend_label='Neutrons', color='orangered', line_width=2)
show(fig)

 NEUT_2THE       


## Calculate X-rays powder diffraction pattern

### Modify job definition object

In [11]:
job_info.pattern_type = "XRAY_2THE"
job_info.print_description()

SRTIO3
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases: SRTIO3                                                                                                                          
Name of phases: SRTIO3                                                                                                                          
Lambda range: (1.9121999740600586, 1.9121999740600586)
Lambda ratio: 1.0
Range 2theta: (10.0, 155.4499969482422)
Range sin(theta)/lambda: (0.05657626688480377, 0.6342995166778564)


### Plot calculated pattern

In [12]:
x_data, y_data_xrays = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_xrays, legend_label='Xrays / 500', color='olivedrab', line_width=2)
show(fig)

 XRAY_2THE       


## Compare neutron and X-ray diffraction patterns

In [13]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons, legend_label='Neutrons', color='orangered', line_width=2)
fig.line(x_data, y_data_xrays/500, legend_label='Xrays / 500', color='olivedrab', line_width=2)
show(fig)

## Calculate neutron powder diffraction pattern (2nd iteration)

### Modify job definition object

In [14]:
job_info.pattern_type = "NEUT_2THE"
job_info.print_description()

SRTIO3
Number of patterns: 1
Type of pattern: NEUT_2THE       
Number of phases: 1
Name of phases: SRTIO3                                                                                                                          
Name of phases: SRTIO3                                                                                                                          
Lambda range: (1.9121999740600586, 1.9121999740600586)
Lambda ratio: 1.0
Range 2theta: (10.0, 155.4499969482422)
Range sin(theta)/lambda: (0.05657626688480377, 0.6342995166778564)


### Plot calculated pattern

In [15]:
x_data, y_data_neutrons2 = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons2, legend_label='Neutrons II', color='orange', line_width=2)
show(fig)

 NEUT_2THE       


## Calculate X-rays powder diffraction pattern (2nd iteration)

### Modify job definition object

In [16]:
job_info.pattern_type = "XRAY_2THE"
job_info.print_description()

SRTIO3
Number of patterns: 1
Type of pattern: XRAY_2THE       
Number of phases: 1
Name of phases: SRTIO3                                                                                                                          
Name of phases: SRTIO3                                                                                                                          
Lambda range: (1.9121999740600586, 1.9121999740600586)
Lambda ratio: 1.0
Range 2theta: (10.0, 155.4499969482422)
Range sin(theta)/lambda: (0.05657626688480377, 0.6342995166778564)


### Plot calculated pattern

In [17]:
x_data, y_data_xrays2 = diffraction_pattern_xy(space_group, unit_cell, atom_list, job_info)
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_xrays2, legend_label='Xrays II / 500', color='darkgreen', line_width=2)
show(fig)

 XRAY_2THE       


## Compare neutron and X-ray diffraction patterns (2nd iteration)

In [18]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons2, legend_label='Neutrons II', color='orange', line_width=2)
fig.line(x_data, y_data_xrays2/500, legend_label='Xrays II / 500', color='darkgreen', line_width=2)
show(fig)

## Compare neutron and X-ray diffraction patterns (all iterations)

In [19]:
fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
fig.line(x_data, y_data_neutrons, legend_label='Neutrons', color='orangered', line_width=2)
fig.line(x_data, y_data_xrays/500, legend_label='Xrays / 500', color='olivedrab', line_width=2)
fig.line(x_data, y_data_neutrons2, legend_label='Neutrons II', color='orange', line_width=2)
fig.line(x_data, y_data_xrays2/500, legend_label='Xrays II / 500', color='darkgreen', line_width=2)
show(fig)